In [1]:
.open tmp3.db
.load ../../dist/vec0
.load ./lembed0

insert into lembed_models(name, model)
  values (
    'default',
    lembed_model_from_file('all-MiniLM-L6-v2.e4ce9877.q8_0.gguf')
  );

select vec_version(), lembed_version();

vec_version(),lembed_version()
v0.1.2-alpha.6,v0.0.1-alpha.8


## FTS Search

In [2]:
.param set query planned parenthood

select
  rowid,
  highlight(fts_headlines, 0, '<b>', '</b>') as headline_highlighted,
  rank
from fts_headlines
where headline match :query
order by rank
limit 10;

rowid,headline_highlighted,rank
4666,Kamala Harris visits <b>Planned</b> <b>Parenthood</b> clinic,-18.9139950477264
6521,Former Marine sentenced to 9 years in prison for firebombing <b>Planned</b> <b>Parenthood</b> clinic,-14.807022703838651


##  Vector Search

In [3]:
.param set query planned parenthood

select
  article_id,
  articles.headline,
  distance
from vec_headlines
left join articles on articles.rowid = vec_headlines.article_id
where headline_embedding match lembed(:query)
  and k = 10;

article_id,headline,distance
9475,Inside a Gen Z campaign to shake up Congress,1.0655490159988403
11236,Bunnie XO and Jelly Roll announce plans to have a baby using IVF,1.071580171585083
4110,First over-the-counter birth control pill hits store shelves,1.0831280946731567
1808,More nonpregnant women are requesting abortion pills to have on hand,1.0897283554077148
11154,Meet the anti-abortion group using white coats and research to advance its cause,1.089759349822998
5980,Deathbed confession leads to bodies of mother and daughter killed 24 years ago,1.092090368270874
3540,"Holding photos of their deceased children, parents lobby Congress to pass online safety legislation",1.1028637886047363
4109,Alabama fertility clinic at the center of IVF ruling is resuming some procedures,1.103183627128601
4889,"Inside the organized crime rings plaguing Ulta, T.J. Maxx, Walgreens and other retailers",1.1056489944458008
86,Florida bars transgender people from changing the sex on their driver's licenses,1.1073163747787476


## FTS + Vector search: RFF

In [4]:
.param set query planned parenthood


.param set weight_fts 1.0
.param set weight_vec 1.0
.param set rrf_k 60
.param set k 10


with vec_matches as (
  select
    article_id,
    row_number() over (order by distance) as rank_number,
    distance
  from vec_headlines
  where
    headline_embedding match lembed(:query)
    and k = :k
  order by distance
),
fts_matches as (
  select
    rowid,
    row_number() over (order by rank) as rank_number,
    rank as score
  from fts_headlines
  where headline match :query
  limit :k
),
final as (
  select
    articles.id,
    articles.headline,
    vec_matches.distance as vector_distance,
    fts_matches.score as fts_score,
    coalesce(1.0 / (:rrf_k + fts_matches.rowid), 0.0) * :weight_fts +
      coalesce(1.0 / (:rrf_k + vec_matches.article_id), 0.0) * :weight_vec
      as combined_score

  from fts_matches
  full outer join vec_matches on vec_matches.article_id = fts_matches.rowid
  join articles on articles.rowid = coalesce(fts_matches.rowid, vec_matches.article_id)
  order by combined_score desc
)
select * from final;



id,headline,vector_distance,fts_score,combined_score
86,Florida bars transgender people from changing the sex on their driver's licenses,1.1073163747787476,,0.00684931506849315
1808,More nonpregnant women are requesting abortion pills to have on hand,1.0897283554077148,,0.0005353319057815846
3540,"Holding photos of their deceased children, parents lobby Congress to pass online safety legislation",1.1028637886047363,,0.0002777777777777778
4109,Alabama fertility clinic at the center of IVF ruling is resuming some procedures,1.103183627128601,,0.00023986567522187575
4110,First over-the-counter birth control pill hits store shelves,1.0831280946731567,,0.00023980815347721823
4666,Kamala Harris visits Planned Parenthood clinic,,-18.9139950477264,0.00021159542953872197
4889,"Inside the organized crime rings plaguing Ulta, T.J. Maxx, Walgreens and other retailers",1.1056489944458008,,0.00020206102242877348
5980,Deathbed confession leads to bodies of mother and daughter killed 24 years ago,1.092090368270874,,0.00016556291390728477
6521,Former Marine sentenced to 9 years in prison for firebombing Planned Parenthood clinic,,-14.807022703838651,0.000151952590791673
9475,Inside a Gen Z campaign to shake up Congress,1.0655490159988403,,0.0001048767697954903


In [ ]:
select json('[1,2,3,4]'), vec_f32(X'AABBCCDD')

In [5]:
select * from bytecode('select count(*) from pragma_table_list;')

addr,opcode,p1,p2,p3,p4,p5,comment,subprog,nexec,ncycle
0,Init,0,12,0,,0,,,0,0
1,Null,0,1,1,,0,,,0,0
2,VOpen,0,0,0,vtab:7F7FCCF08AE0,0,,,0,0
3,Integer,0,2,0,,0,,,0,0
4,Integer,0,3,0,,0,,,0,0
5,VFilter,0,8,2,,0,,,0,0
6,AggStep,0,0,1,count(0),0,,,0,0
7,VNext,0,6,0,,0,,,0,0
8,AggFinal,1,0,0,count(0),0,,,0,0
9,Copy,1,4,0,,0,,,0,0
